In [ ]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
import theano.d3viz as d3v
from IPython.display import IFrame
from theano_models.visualization import d3viz

In [ ]:
from theano_models.postmaps import reduce_postmap
from theano_models import Subgraph, clone_renew_rng, inputting_references, outputting_references

In [ ]:
inputting_references.add('loss_inputs')
outputting_references.add('loss')

In [ ]:
import numpy as np

In [ ]:
import theano.tensor as T
import theano
from theano.printing import Print, debugprint
from theano.tensor.shared_randomstreams import RandomStreams

# simple reduce

In [ ]:
summedup = theano.reduce(lambda a, b: a + b, [i], np.array(0, i.dtype))[0]

In [ ]:
f = theano.function([i], summedup)

In [ ]:
f([1,5,2])

# complex reduce

In [ ]:
rng = RandomStreams()
a = rng.random_integers()
i = T.dvector("i")
# expr = Print("a1")(a) + T.sum(i)
expr = (a) + T.sum(i)

In [ ]:
model = Subgraph(loss_inputs=[i], loss=expr, name="single")
model

In [ ]:
rmodel = Subgraph(reduce_postmap(model), name="multiple")
rmodel

In [ ]:
f = rmodel['loss_inputs'], rmodel['loss']
d3viz(f, 'tmp/rloss.html')
IFrame('tmp/rloss.html', width=700, height=500)

In [ ]:
f = theano.function(model['loss_inputs'], model['loss'], mode="FAST_RUN", profile=True)
rf = theano.function(rmodel['loss_inputs'], rmodel['loss'], mode="FAST_RUN", profile=True)

In [ ]:
%%timeit
for row in [[1]*1000]*1000:
    f(row)

In [ ]:
%timeit rf([[1]*1000]*1000)

In [ ]:
d3viz(f, 'tmp/floss.html', match_by_names=True)
IFrame('tmp/floss.html', width=700, height=500)

In [ ]:
d3viz(rf, 'tmp/rfloss.html', match_by_names=True)
IFrame('tmp/rfloss.html', width=700, height=500)

# indendent subgraphs

In [26]:
import theano
import theano.tensor as T
from theano.printing import debugprint
from theano.tensor.shared_randomstreams import RandomStreams


from theano_models import util

from theano_models.util.theano_helpers import independent_subgraphs

a, b, c = T.dvectors("abc")
_a, _b, _c = a.sum(), b.sum(), c.sum()
_a.name, _b.name, _c.name = "_a", "_b", "_c"
_1 = _a +_b
_1.name = "1"
_2 = _c + _a
_2.name = "2"
expr = _1 + _2

sub1, sub2 = independent_subgraphs([a,b], [c], expr)
sub1, sub2

([1, _a], [_c])

In [44]:
f = theano.function([a,b,c], expr)
f1 = theano.function([a,b], sub1)
f1_ = theano.function([a,b], [a,b] + sub1)
f2 = theano.function([c, a,b] + sub1, expr, on_unused_input="ignore")
f2_ = theano.function([c] + sub1, expr)

In [30]:
test = range(50)

In [45]:
%%timeit
for i in test:
    f(test,test,test)

100 loops, best of 3: 2.5 ms per loop


In [46]:
%%timeit
h = f1(test,test)

10000 loops, best of 3: 36.3 µs per loop


In [50]:
%%timeit
h = f1(test,test)
for i in test:
    f2(test,test,test, *h)

100 loops, best of 3: 2.77 ms per loop


This does not seem to add any improvements at all, despit the calculations done in h are quite extensive

In [48]:
%%timeit
h = f1(test,test)
for i in test:
    f2_(test, *h)

1000 loops, best of 3: 1.36 ms per loop


However without on_unused_input="ignore" everything works just fine

In [51]:
%%timeit
h = f1_(test,test)
for i in test:
    f2(test, *h)

1000 loops, best of 3: 1.56 ms per loop


Even with just including them in the default version, the speed is much better. So this should get the default if MissingInputError appears